<h1><i>kiara</i>: Natural Language Processing (NLP)</h1>

Welcome back! Now that we're comfortable with what *kiara* looks like and what it can do to help track your data and your research process, let's try out some of the digital analysis tools, starting with **Natural Language Processing**.

<span style ="color:blue">Lorella could there be some general information here on language processing/point to some literature maybe?</span>
<h1>Why NLP?</h1>

First of all, why bothering with NLP? Natural language processing technology allows researchers to sort through unstructured data such as plain text. In other words, by adding numerical value to text, computers can <i>understand</i> language and perform advanced operations such as text categorisation, labelling, summarisation and so on.
There are two main stages in NLP: pre-processing and analysis (aka, algorithm development and/or implementation). Here we cover both stages through the example of some of the most common pre-processing operations such as tokenisation, lowercasing, removing stopwords etc. in the first part. For the second part, we will use the example of another widely used text analysis method called topic modelling.
For more information about the pre-processing operations and topic modelling and a more in-depth discussion particularly for humanities research, please refer to this repository [here](https://github.com/DHARPA-Project/TopicModelling-).

<h3>Starting the Process</h3>

Let's start by double checking that we have all the required plugins, and setting up an API for us to use *kiara*. We'll do this all in one go this time, but if you're unsure, feel free to head back to the [installation notebook](http://dharpa.org/kiara.documentation/latest/workshop/workshop/) to look over this section again.

In [1]:
try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()

from kiara import KiaraAPI
kiara = KiaraAPI.instance()

Now we're all set up, we want to download some text to work with in our language processing analyis. <br/>
For our example here we will be using a relatively small number of texts. This is a sample taken from the larger corpus [*ChroniclItaly 3.0*](http://doi.org/10.5281/zenodo.4596345) (Viola and Fiscarelli 2021, [Viola 2021](https://www.euppublishing.com/doi/full/10.3366/ijhac.2021.0268)), an open access digital heritage collection of Italian immigrant newspapers published in the United States from 1898 to 1936.
The corpus that we use here includes the digitized (OCRed) front pages of the Italian language newspaper *La rassegna* as collected from [*Chronicling America*](https://chroniclingamerica.loc.gov/newspapers/), an Internet-based, searchable database of U.S. newspapers published in the United States from 1789 to 1963 made available by the Library of Congress.
These files are also a good examples because their filenames already contain important metadata information such as the publication date. The file name structure is: LCCNnumber_date_pageNumber_ocr.txt. Therefore, the file name ‘sn84037025_1917-04-14_ed-1_seq-1_ocr.txt ’ refers to the OCR text file of the first page of the first edition of *La Rassegna* published on 14 April 1917. *kiara* allows us to retrieve both the files and the metadata in the filenames. This is very useful for historical research, but also to keep track of how we are intervening on our sources. Let's see how this works.

In [2]:
kiara.list_operation_ids('download')

['download.file', 'download.file_bundle']

Last time we only wanted one file, but with language processing we might want a bigger corpus. <br/>
Let's have a look at `download.file_bundle` this time.

In [3]:
kiara.retrieve_operation_info('download.file_bundle')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         onboarding                                                                                                
                     Labels       package: kiara_plugin.onboarding                                                                          
                     References   source_repo: ]8;id=705361;https://github.com/DHARPA-Project/kiara_plugin.onboarding\https://github.com/DHARPA-Project/kiara_plugin.onboarding]8;;\                                    
                                  documentation: ]8;id=723026;https://DHARPA-Project.github.io/kiara_plugin.onboarding/\https://DHARPA-Project.github.io/kiara_plugin.onboarding/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   -- n/a --                                                                                              
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type     description                                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       url          string   The url of an archive/zip file to download.    yes        -- no default --     
                                       sub_path     string   A relative path to select only a sub-folder    no         -- no default --     
                                                             from the archive.                                                              
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name          type          description                                                        
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       file_bundle         file_bundle   The downloaded file bundle.                                        
                                       download_metadata   dict          Metadata about the download.                                       
                                                                                                                                            
                  

So we still want a url, but for a zip file that we can download.
Here's some example data for us to use.

Again, we need to define the <span style="color:green">inputs</span>, use `kiara.run_job` with our chosen operation `download.file_bundle` and store this as our <span style="color:red">outputs</span>.

In [4]:
inputs = {
    "url": "https://github.com/DHARPA-Project/kiara.examples/archive/refs/heads/main.zip",
    "sub_path": "kiara.examples-main/examples/data/text_corpus/data"
 }

outputs = kiara.run_job('download.file_bundle', inputs=inputs)
outputs


patool: Extracting /tmp/tmpetdyl6jt ...
patool: ... /tmp/tmpetdyl6jt extracted to `/tmp/tmppm5ttrt8'.


╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field               value                                                                                                              │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   download_metadata                                                                                                                      │
│                         dict data     {                                                                                                  │
│                                         "response_headers": [                                                                            │
│                                           {                                                                                              │
│                                             "access-control-allow-origin": "https://render.githubusercontent.com",                       │
│                                             "content-disposition": "attachment; filename=kiara.examples-main.zip",                       │
│                                             "content-security-policy": "default-src 'none'; style-src 'unsafe-inline'; sandbox",         │
│                                             "content-type": "application/zip",                                                           │
│                                             "etag": "W/\"d6648e3a3d05bead78bcb3baef700b3f0348cf08d404199408e4c5f12ae9607e\"",            │
│                                             "strict-transport-security": "max-age=31536000",                                             │
│                                             "vary": "Authorization,Accept-Encoding,Origin",                                              │
│                                             "x-content-type-options": "nosniff",                                                         │
│                                             "x-frame-options": "deny",                                                                   │
│                                             "x-xss-protection": "1; mode=block",                                                         │
│                                             "date": "Mon, 13 Feb 2023 16:28:11 GMT",                                                     │
│                                             "transfer-encoding": "chunked",                                                              │
│                                             "x-github-request-id": "CA08:B383:4B5AFA:6096CC:63EA651A"                                    │
│                                           },                                                                                             │
│                                           {                                                                                              │
│                                             "server": "GitHub.com",                                                                      │
│                                             "date": "Mon, 13 Feb 2023 16:27:31 GMT",                                                     │
│                                             "content-type": "text/html; charset=utf-8",                                                  │
│                                             "vary": "X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X…     │
│                                             "location": "https://codeload.github.com/DHARPA-Project/kiara.examples/zip/refs/heads/m…     │
│                                             "cache

Great, we've successfully imported a bundle of files this time rather than just one. This has given us both the metadata for the files, and the files themselves. As you can see, *kiara* also gives us additional information on the composition of the text files, that is the number of tokens. This information will be useful later when we will intervene on these files to keep track of how we have changed them. For now, let's save the files in a separate variable for us to use later.

In [5]:
file_bundle = outputs['file_bundle']

<h3>Preparing the Texts</h3>

Now that we have imported the files, let's give them some structure. For this, we will need the `create.table.from.file_bundle` function (similar to the [installation notebook](https://github.com/DHARPA-Project/kiara.documentation/blob/develop/docs/workshop/workshop.ipynb) which you are welcome to revisit at any time). Let's have a look by exploring the list of avaibale operations.

In [6]:
kiara.retrieve_operation_info('create.table.from.file_bundle')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=424203;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=610726;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a table value from a text file_bundle.                                                          
                                                                                                                                            
                                     The resulting table will have (at a minimum) the following collumns:                                   
                                     - id: an auto-assigned index                                                                           
                                     - rel_path: the relative path of the file (from the provided base path)                                
                                     - content: the text file content                                                                       
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name    type          description                              Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       file_bundle   file_bundle   The source value (of type                yes        -- no default --     
                                                                   'file_bundle').                                                          
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name   type    description                                                                     
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       t

Let's use the file bundle we downloaded earlier and saved in our variable, and run this *kiara* table function.

In [7]:
inputs = {
    'file_bundle' : file_bundle
}

outputs = kiara.run_job('create.table.from.file_bundle', inputs=inputs)
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   table                                                                                                                                  │
│             id   rel_path                        mime_type    size    content                          file_name                         │
│            ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│             0    La_Ragione/sn84037024_1917-04   text/plain   16613   LA RAGIONE                       sn84037024_1917-04-25_ed-1_se     │
│             1    La_Ragione/sn84037024_1917-04   text/plain   16679   LA RAG ONE                       sn84037024_1917-04-25_ed-2_se     │
│             2    La_Ragione/sn84037024_1917-04   text/plain   16793   LA RAGIONE                       sn84037024_1917-04-25_ed-3_se     │
│             3    La_Ragione/sn84037024_1917-04   text/plain   16235   contro i vili, i camorristi, i   sn84037024_1917-04-25_ed-4_se     │
│             4    La_Ragione/sn84037024_1917-05   text/plain   18346   contro i vili, i camorristi, i   sn84037024_1917-05-05_ed-1_se     │
│             5    La_Ragione/sn84037024_1917-05   text/plain   18474   LA RAGIONA                       sn84037024_1917-05-05_ed-2_se     │
│             6    La_Ragione/sn84037024_1917-05   text/plain   18280   LA RAGIONE                       sn84037024_1917-05-05_ed-3_se     │
│             7    La_Ragione/sn84037024_1917-05   text/plain   18481   LA RAGIONE                       sn84037024_1917-05-05_ed-4_se     │
│             8    La_Ragione/sn84037024_1917-05   text/plain   18620   contro i vili, i camorristi, i   sn84037024_1917-05-16_ed-1_se     │
│             9    La_Ragione/sn84037024_1917-05   text/plain   18698   LA RAG ONE                       sn84037024_1917-05-16_ed-2_se     │
│             10   La_Ragione/sn84037024_1917-05   text/plain   18540   contro 1 vili, i camorristi, i   sn84037024_1917-05-16_ed-3_se     │
│             11   La_Rassegna/sn84037025_1917-0   text/plain   19397   ■■■                              sn84037025_1917-04-07_ed-1_se     │
│             12   La_Rassegna/sn84037025_1917-0   text/plain   20647   La Rassegna                      sn84037025_1917-04-14_ed-1_se     │
│             13   La_Rassegna/sn84037025_1917-0   text/plain   20650   Both Phones                      sn84037025_1917-04-14_ed-2_se     │
│             14   La_Rassegna/sn84037025_1917-0   text/plain   21017   ■ jSrìt** W?? iIK 38®f- i^M      sn84037025_1917-04-21_ed-1_se     │
│             15   La_Rassegna/sn84037025_1917-0   text/plain   20982   ■Both Phones                     sn84037025_1917-04-21_ed-2_se     │
│                                                                                                                                          │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Great, this has taken all the information from the files we downloaded and made it a bit easier to navigate. In order to process and analyse our sources, we need to work with the files' content which is in the column 'content'. Let's run `kiara.list_operation_ids('table')' to see how we might be able to do that.

In [8]:
kiara.list_operation_ids('table')

['create.database.from.table',
 'create.network_data.from.tables',
 'create.table.from.file',
 'create.table.from.file_bundle',
 'export.table.as.csv_file',
 'extract.date_array.from.table',
 'import.table.from.local_file_path',
 'import.table.from.local_folder_path',
 'query.table',
 'table.pick.column',
 'table_filter.drop_columns',
 'table_filter.select_columns',
 'table_filter.select_rows']

As we are interested in one column, the `table.pick.column` operation seems like a good fit.

In [9]:
kiara.retrieve_operation_info('table.pick.column')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=490744;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=357607;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Pick one column from a table, returning an array.                                                      
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name    type     description                                   Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       table         table    A table.                                      yes        -- no default --     
                                       column_name   string   The name of the column to extract.            yes        -- no default --     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name   type    description                                                                     
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       array        array   The column.                                                                     
                                                                                                                                            
                  

So here we need two <span style="color:green">inputs</span>, the **table** we just made and the name of the **coloumn** we want to pick. 

Let's specify our outputs again and run the function. In this way, we retain the content of the files as the variable we need for NLP.

In [10]:
inputs = {
    'table' : outputs['table'],
    'column_name' : 'content'
}

outputs = kiara.run_job('table.pick.column', inputs=inputs)
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────                                      │
│   array                                                                                                                                  │
│             LA RAGIONE                                                                                                                   │
│             LA RAG ONE                                                                                                                   │
│             LA RAGIONE                                                                                                                   │
│             contro i vili, i camorristi, i sicari, i falsari e gli austriacanti, nemici dell ...                                         │
│             contro i vili, i camorristi, i sicari, i falsari e gli austriacanti, nemici dell ...                                         │
│             LA RAGIONA                                                                                                                   │
│             LA RAGIONE                                                                                                                   │
│             LA RAGIONE                                                                                                                   │
│             contro i vili, i camorristi, i sicari, i falsari e gli austriacanti, nemici dell ...                                         │
│             LA RAG ONE                                                                                                                   │
│             contro 1 vili, i camorristi, i sicari, i falsari e gli austriacanti, nemici dell ...                                         │
│             ■■■                                                                                                                          │
│             La Rassegna                                                                                                                  │
│             Both Phones                                                                                                                  │
│             ■ jSrìt** W?? iIK 38®f- i^M                                                                                                  │
│             ■Both Phones                                                                                                                 │
│                                                                                                                                          │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<h3>Language Processing and Topic Modelling</h3>

Great, we've downloaded and prepped our material for text analysis. Not we want to see what kind of functions are included in *kiara* for language processing. Let's have a look and see what's included in the `kiara_plugin.language_processing` package.

<span style="color:blue">Lorella this may/may not need some extra description on what the different operations do</span>

In [11]:
infos = metadata = kiara.retrieve_operations_info()
operations = {}
for op_id, info in infos.item_infos.items():
    if info.context.labels.get("package", None) == "kiara_plugin.language_processing":
        operations[op_id] = info

print(operations.keys())

dict_keys(['create.stopwords_list', 'generate.LDA.for.tokens_array', 'preprocess.tokens_array', 'remove_stopwords.from.tokens_array', 'tokenize.string', 'tokenize.texts_array'])


The contents of our text files have been stored as an **array**, so let's start by tokenizing this using the `tokenize.texts_array` function.

In [12]:
kiara.retrieve_operation_info('tokenize.texts_array')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         language_processing, tokenize, tokens                                                                     
                     Labels       package: kiara_plugin.language_processing                                                                 
                     References   source_repo: ]8;id=826757;https://github.com/DHARPA-Project/kiara_plugin.language_processing\https://github.com/DHARPA-Project/kiara_plugin.language_processing]8;;\                           
                                  documentation: ]8;id=434802;https://DHARPA-Project.github.io/kiara_plugin.language_processing/\https://DHARPA-Project.github.io/kiara_plugin.language_processing/]8;;\                         
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Split sentences into words or words into characters.                                                   
                                                                                                                                            
                                     In other words, this operation establishes the word boundaries (i.e., tokens) a very helpful way of    
                                     finding patterns. It is also the typical step prior to stemming and lemmatization                      
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name         type      description                             Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       texts_array        array     An array of text items to be            yes        -- no default --     
                                                                    tokenized.                                                              
                                       tokenize_by_word   boolean   Whether to tokenize by word             no         True                 
                                                                    (default), or character.                                                
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name     type    description                                                                   
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                

Great, let's give it a go!

In [13]:
inputs = {
    'texts_array': outputs['array']
}

outputs = kiara.run_job('tokenize.texts_array', inputs=inputs)
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────                               │
│   tokens_array                                                                                                                           │
│                    ['LA', 'RAGIONE', 'ORGANO', 'DI', 'DIFESA', 'DELLA', "ITALIANITÀ'", 'contro', '1 ...                                  │
│                    ['LA', 'RAG', 'ONE', 'contro', 'i', 'vili', ',', 'i', 'camorristi', ',', 'i', 's ...                                  │
│                    ['LA', 'RAGIONE', 'ORGANO', 'DI', 'DIFESA', 'DELLA', 'ITALIANITÀ', 'contro', 'i' ...                                  │
│                    ['contro', 'i', 'vili', ',', 'i', 'camorristi', ',', 'i', 'sicari', ',', 'i', 'f ...                                  │
│                    ['contro', 'i', 'vili', ',', 'i', 'camorristi', ',', 'i', 'sicari', ',', 'i', 'f ...                                  │
│                    ['LA', 'RAGIONA', 'ORGANO', 'DI', 'DIFESA', 'DELLA', 'ITALIANITÀ', 'contro', 'i' ...                                  │
│                    ['LA', 'RAGIONE', 'ORGANO', 'DI', 'DIFESA', 'DELLA', "ITALIANITÀ'", 'contro', 'i ...                                  │
│                    ['LA', 'RAGIONE', 'contro', 'i', 'vili', ',', '1', 'camorristi', ',', 'i', 'sica ...                                  │
│                    ['contro', 'i', 'vili', ',', 'i', 'camorristi', ',', 'i', 'sicari', ',', 'i', 'f ...                                  │
│                    ['LA', 'RAG', 'ONE', 'ORGANO', 'DI', 'DIFESA', 'DELLA', 'ITALIANITÀ', "''", 'con ...                                  │
│                    ['contro', '1', 'vili', ',', 'i', 'camorristi', ',', 'i', 'sicari', ',', 'i', 'f ...                                  │
│                    ['■■■', 'La', 'Rassegna', '_', 'I', 'Both', 'Phones', 'ANNO', 'L', 'No', '.', '1 ...                                  │
│                    ['La', 'Rassegna', 'Jjoth', 'Phones', 'ANNO', 'L', 'No', '.', '2', 'BASTA', '!', ...                                  │
│                    ['Both', 'Phones', 'ANNO', 'I', '.', 'No', '.', '2', 'BASTA', '!', '...', 'uà',  ...                                  │
│                    ['■', 'jSrìt', '*', '*', 'W', '?', '?', 'iIK', '38®f-', 'i^M', 'F', '<', '5É', ' ...                                  │
│                    ['■Both', 'Phones', 'ANNO', '11', '.', 'No', '.', '5', 'LE', 'COSE', 'A', 'POSTO ...                                  │
│                                                                                                                                          │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

We can see from the printed preview that this has tokenized the contents for each of the text files we imported.

Now we can work on pre-processing some of this text. Let's look at what options we have in the `preprocess.tokens_array` operation.

In [14]:
kiara.retrieve_operation_info('preprocess.tokens_array')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         language_processing, tokens, preprocess                                                                   
                     Labels       package: kiara_plugin.language_processing                                                                 
                     References   source_repo: ]8;id=669493;https://github.com/DHARPA-Project/kiara_plugin.language_processing\https://github.com/DHARPA-Project/kiara_plugin.language_processing]8;;\                           
                                  documentation: ]8;id=210446;https://DHARPA-Project.github.io/kiara_plugin.language_processing/\https://DHARPA-Project.github.io/kiara_plugin.language_processing/]8;;\                         
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Preprocess lists of tokens, incl. lowercasing, remove special characers, etc.                          
                                                                                                                                            
                                     Lowercasing: Lowercase the words. This operation is a double-edged sword. It can be effective at       
                                     yielding potentially better results in the case of relatively small datasets or datatsets with a       
                                     high percentage of OCR mistakes. For instance, if lowercasing is not performed, the algorithm will     
                                     treat USA, Usa, usa, UsA, uSA, etc. as distinct tokens, even though they may all refer to the same     
                                     entity. On the other hand, if the dataset does not contain such OCR mistakes, then it may become       
                                     difficult to distinguish between homonyms and make interpreting the topics much harder.                
                                                                                                                                            
                                     Removing stopwords and words with less than three characters: Remove low information words. These      
                                     are typically words such as articles, pronouns, prepositions, conjunctions, etc. which are not         
                                     semantically salient. There are numerous stopword lists available for many, though not all,            
                                     languages which can be easily adapted to the individual researcher's needs. Removing words with less   
                                     than three characters may additionally remove many OCR mistakes. Both these operations have the dual   
                                     advantage of yielding more reliable results while reducing the size of the dataset, thus in turn       
                                     reducing the required processing power. This step can therefore hardly be considered optional in TM.   
                                                                                                                                            
                

There's lots of options here to choose from to help pre-process our texts.

Let's start by removing a custom **stopwords list**. Feel free to have a play around, and add and change some of the words!

First, we need to create the **stopwords list**, and there is an operation for that:

In [15]:
kiara.retrieve_operation_info('create.stopwords_list')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         language_processing                                                                                       
                     Labels       package: kiara_plugin.language_processing                                                                 
                     References   source_repo: ]8;id=417009;https://github.com/DHARPA-Project/kiara_plugin.language_processing\https://github.com/DHARPA-Project/kiara_plugin.language_processing]8;;\                           
                                  documentation: ]8;id=119074;https://DHARPA-Project.github.io/kiara_plugin.language_processing/\https://DHARPA-Project.github.io/kiara_plugin.language_processing/]8;;\                         
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a list of stopwords from one or multiple sources.                                               
                                                                                                                                            
                                     This will download nltk stopwords if necessary, and merge all input lists into a single, sorted list   
                                     without duplicates.                                                                                    
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name      type   description                                   Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       languages       list   A list of languages, will be used to          no         -- no default --     
                                                              retrieve language-specific stopword from                                      
                                                              nltk.                                                                         
                                       stopword_list   list   A list of additional, custom stopwords.       no         -- no default --     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name       type   description                                                                  
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                

Let's say we trust the default 'italian' stopword list that comes with nltk, but want to augment it with a few of our custom stopwords. We'd do it like this:

In [16]:
custom_stopword_list = ['la', 'i']

inputs = {
    "languages": ["italian"],
    "stopword_list": custom_stopword_list
}

stopwords_outputs = kiara.run_job('create.stopwords_list', inputs=inputs)
my_stopwords_list = stopwords_outputs['stopwords_list']
my_stopwords_list.data

ListModel(list_data=['a', 'abbia', 'abbiamo', 'abbiano', 'abbiate', 'ad', 'agl', 'agli', 'ai', 'al', 'all', 'alla', 'alle', 'allo', 'anche', 'avemmo', 'avendo', 'avesse', 'avessero', 'avessi', 'avessimo', 'aveste', 'avesti', 'avete', 'aveva', 'avevamo', 'avevano', 'avevate', 'avevi', 'avevo', 'avrai', 'avranno', 'avrebbe', 'avrebbero', 'avrei', 'avremmo', 'avremo', 'avreste', 'avresti', 'avrete', 'avrà', 'avrò', 'avuta', 'avute', 'avuti', 'avuto', 'c', 'che', 'chi', 'ci', 'coi', 'col', 'come', 'con', 'contro', 'cui', 'da', 'dagl', 'dagli', 'dai', 'dal', 'dall', 'dalla', 'dalle', 'dallo', 'degl', 'degli', 'dei', 'del', 'dell', 'della', 'delle', 'dello', 'di', 'dov', 'dove', 'e', 'ebbe', 'ebbero', 'ebbi', 'ed', 'era', 'erano', 'eravamo', 'eravate', 'eri', 'ero', 'essendo', 'faccia', 'facciamo', 'facciano', 'facciate', 'faccio', 'facemmo', 'facendo', 'facesse', 'facessero', 'facessi', 'facessimo', 'faceste', 'facesti', 'faceva', 'facevamo', 'facevano', 'facevate', 'facevi', 'facevo', 'fai

In [17]:
inputs = {
    'tokens_array': outputs['tokens_array'],
    'remove_stopwords' : my_stopwords_list
}

outputs = kiara.run_job('preprocess.tokens_array', inputs=inputs)
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────                               │
│   tokens_array                                                                                                                           │
│                    ['RAGIONE', 'ORGANO', 'DIFESA', "ITALIANITÀ'", '1', 'vili', ',', 'camorristi', ' ...                                  │
│                    ['RAG', 'ONE', 'vili', ',', 'camorristi', ',', 'sicari', ',', 'falsari', 'austri ...                                  │
│                    ['RAGIONE', 'ORGANO', 'DIFESA', 'ITALIANITÀ', 'vili', ',', 'camorristi', ',', 's ...                                  │
│                    ['vili', ',', 'camorristi', ',', 'sicari', ',', 'falsari', 'austriacanti', ',',  ...                                  │
│                    ['vili', ',', 'camorristi', ',', 'sicari', ',', 'falsari', 'austriacanti', ',',  ...                                  │
│                    ['RAGIONA', 'ORGANO', 'DIFESA', 'ITALIANITÀ', 'vili', ',', 'camorristi', ',', 's ...                                  │
│                    ['RAGIONE', 'ORGANO', 'DIFESA', "ITALIANITÀ'", 'vili', ',', 'camorristi', ',', ' ...                                  │
│                    ['RAGIONE', 'vili', ',', '1', 'camorristi', ',', 'sicari', ',', 'falsari', 'aust ...                                  │
│                    ['vili', ',', 'camorristi', ',', 'sicari', ',', 'falsari', 'austriacanti', ',',  ...                                  │
│                    ['RAG', 'ONE', 'ORGANO', 'DIFESA', 'ITALIANITÀ', "''", 'vili', ',', 'camorristi' ...                                  │
│                    ['1', 'vili', ',', 'camorristi', ',', 'sicari', ',', 'falsari', 'austriacanti',  ...                                  │
│                    ['■■■', 'Rassegna', '_', 'Both', 'Phones', 'ANNO', 'No', '.', '1', 'perche', "'" ...                                  │
│                    ['Rassegna', 'Jjoth', 'Phones', 'ANNO', 'No', '.', '2', 'BASTA', '!', '...', 'qu ...                                  │
│                    ['Both', 'Phones', 'ANNO', '.', 'No', '.', '2', 'BASTA', '!', '...', 'uà', 'quai ...                                  │
│                    ['■', 'jSrìt', '*', '*', 'W', '?', '?', 'iIK', '38®f-', 'i^M', 'F', '<', '5É', ' ...                                  │
│                    ['■Both', 'Phones', 'ANNO', '11', '.', 'No', '.', '5', 'COSE', 'POSTO', 'va', 'd ...                                  │
│                                                                                                                                          │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Great. Let's take this a bit further and try and combine two of our options in one function. In reality, we can add all the <span style="color:green">inputs</span> together in one job, but let's start with converting everything into lowercase and removing any words with non-alphanumeric symbols.

In [18]:
inputs = {
    'tokens_array': outputs['tokens_array'],
    'to_lowercase' : True,
    'remove_non_alpha' : True
}

outputs = kiara.run_job('preprocess.tokens_array', inputs=inputs)
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────                               │
│   tokens_array                                                                                                                           │
│                    ['ragione', 'organo', 'difesa', 'vili', 'camorristi', 'sicari', 'falsari', 'aust ...                                  │
│                    ['rag', 'one', 'vili', 'camorristi', 'sicari', 'falsari', 'austriacanti', 'nemic ...                                  │
│                    ['ragione', 'organo', 'difesa', 'italianità', 'vili', 'camorristi', 'sicari', 'f ...                                  │
│                    ['vili', 'camorristi', 'sicari', 'falsari', 'austriacanti', 'nemici', 'patria',  ...                                  │
│                    ['vili', 'camorristi', 'sicari', 'falsari', 'austriacanti', 'nemici', 'patria',  ...                                  │
│                    ['ragiona', 'organo', 'difesa', 'italianità', 'vili', 'camorristi', 'sicari', 'f ...                                  │
│                    ['ragione', 'organo', 'difesa', 'vili', 'camorristi', 'sicari', 'falsari', 'aust ...                                  │
│                    ['ragione', 'vili', 'camorristi', 'sicari', 'falsari', 'austriacanti', 'nemici', ...                                  │
│                    ['vili', 'camorristi', 'sicari', 'falsari', 'austriacanti', 'nemici', 'patria',  ...                                  │
│                    ['rag', 'one', 'organo', 'difesa', 'italianità', 'vili', 'camorristi', 'sicari', ...                                  │
│                    ['vili', 'camorristi', 'sicari', 'falsari', 'austriacanti', 'nemici', 'patria',  ...                                  │
│                    ['rassegna', 'both', 'phones', 'anno', 'no', 'perche', 'de', 'rassegna', 'coloni ...                                  │
│                    ['rassegna', 'jjoth', 'phones', 'anno', 'no', 'basta', 'qualche', 'tempo', 'arte ...                                  │
│                    ['both', 'phones', 'anno', 'no', 'basta', 'uà', 'quaiene', 'tempo', 'parie', 'ua ...                                  │
│                    ['jsrìt', 'w', 'iik', 'f', 'v', 'ht', 'p', 't', 'both', 'phones', 'anno', 'no',  ...                                  │
│                    ['phones', 'anno', 'no', 'cose', 'posto', 'va', 'dicendo', 'va', 'susurran', 'do ...                                  │
│                                                                                                                                          │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Now that we're happy with our prepared pre-processed texts, we can use `generate.LDA.for.tokens_array` to try out some topic modelling. The default for topics is set at seven, but just like the `preprocess.tokens_array` operation, we can play around with the options. Let's have a look.

In [19]:
kiara.retrieve_operation_info('generate.LDA.for.tokens_array')

Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         language_processing, LDA, tokens                                                                          
                     Labels       package: kiara_plugin.language_processing                                                                 
                     References   source_repo: ]8;id=93111;https://github.com/DHARPA-Project/kiara_plugin.language_processing\https://github.com/DHARPA-Project/kiara_plugin.language_processing]8;;\                           
                                  documentation: ]8;id=657156;https://DHARPA-Project.github.io/kiara_plugin.language_processing/\https://DHARPA-Project.github.io/kiara_plugin.language_processing/]8;;\                         
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Perform Latent Dirichlet Allocation on a tokenized corpus.                                             
                                                                                                                                            
                                     This module computes models for a range of number of topics provided by the user.                      
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name          type      description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       tokens_array        array     The text corpus.                       yes        -- no default --     
                                       num_topics_min      integer   The minimal number of topics.          no         7                    
                                       num_topics_max      integer   The max number of topics.              no         7                    
                                       compute_coherence   boolean   Whether to compute the coherence       no         False                
                                                                     score for each model.                                                  
                                       words_per_topic     integer   How many words per topic to put in     no         10                   
                                                                     the result model.                                                      
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                 

We'll stick with the default for now, and generate some topics for our text.

In [20]:
inputs = {
    'tokens_array' : outputs['tokens_array']
}

outputs = kiara.run_job('generate.LDA.for.tokens_array', inputs=inputs)
outputs

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field             value                                                                                                                │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   coherence_map                                                                                                                          │
│                       dict data     {}                                                                                                   │
│                       dict schema   {                                                                                                    │
│                                       "title": "dict",                                                                                   │
│                                       "type": "object"                                                                                   │
│                                     }                                                                                                    │
│                                                                                                                                          │
│   coherence_table   -- none/not set --                                                                                                   │
│   topic_models                                                                                                                           │
│                       dict data     {                                                                                                    │
│                                       "7": [                                                                                             │
│                                         [                                                                                                │
│                                           0,                                                                                             │
│                                           "0.005*\"no\" + 0.004*\"perchè\" + 0.004*\"j\" + 0.004*\"sempre\" + 0.004*\"degenerato\" …     │
│                                         ],                                                                                               │
│                                         [                                                                                                │
│                                           1,                                                                                             │
│                                           "0.004*\"perchè\" + 0.004*\"no\" + 0.004*\"j\" + 0.004*\"degenerato\" + 0.003*\"essi\" + …     │
│                                         ],                                                                                               │
│                                         [                                                                                                │
│                                           2,                                                                                             │
│                                           "0.006*\"no\" + 0.004*\"sempre\" + 0.004*\"j\" + 0.004*\"degenerato\" + 0.003*\"perchè\" …     │
│                                         ],                                                                                               │
│                                         [                                                                                                │
│                                           3,      

<h3>Recording and Tracing our Data</h3>

We've successfully downloaded, organised and pre-processed our text files, and now generated some topics for it. <br/>
Fantastic!

As we know, this means we've made lots of decisions about our research process and our data. But by using *kiara*, we can trace what's changed and the decisions we've made. Let's have a look!


<span style="color:blue">As with the installation notebook, not much to see here yet but will be updated as changes come. Would potentially be useful with operations that require options (like the preproccessing) to know whether this has been selected or not?</span>

In [21]:
topics = outputs['topic_models']

topics.lineage

generate.LDA.for.tokens_array
├── input: compute_coherence (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
├── input: num_topics_max (integer) = fa52d956-1877-4563-89a1-3e4c74550455
├── input: num_topics_min (integer) = fa52d956-1877-4563-89a1-3e4c74550455
├── input: tokens_array (array) = d81a8c7f-5c40-44bc-944a-db84cb62abff
│   └── preprocess.tokens_array
│       ├── input: remove_all_numeric (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
│       ├── input: remove_alphanumeric (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
│       ├── input: remove_non_alpha (boolean) = bc5bf0e1-36bf-4c13-8c2b-2f09bd83d121
│       ├── input: remove_short_tokens (integer) = 61b1e643-b722-4acb-b01e-e6a832224dbf
│       ├── input: remove_stopwords (list) = 855f907b-c13f-4fe9-8cab-47868106d199
│       ├── input: to_lowercase (boolean) = 15b68ab8-a276-4758-a3ec-724e952195f6
│       └── input: tokens_array (array) = 35509c97-903d-474e-8f46-e93c35c6a094
│           └── preprocess.tokens_array
│               ├── input: remove_all_numeric (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
│               ├── input: remove_alphanumeric (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
│               ├── input: remove_non_alpha (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
│               ├── input: remove_short_tokens (integer) = 61b1e643-b722-4acb-b01e-e6a832224dbf
│               ├── input: remove_stopwords (list) = 144578c8-4397-4bfe-bd4d-649174f70784
│               │   └── create.stopwords_list
│               │       ├── input: languages (list) = 15cb527d-51ac-4120-9a6b-09b6a4c990a6
│               │       └── input: stopword_list (list) = b64323c4-d9d7-4a5c-bfe9-d426a08d37df
│               ├── input: to_lowercase (boolean) = 55b4124c-6e21-4357-8572-3cea755e915b
│               └── input: tokens_array (array) = 18b9c947-4d27-482b-935d-d1d59a09eb4d
│                   └── tokenize.texts_array
│                       ├── input: texts_array (array) = eb9c18d1-b991-40eb-8a33-262d4a178059
│                       │   └── table.pick.column
│                       │       ├── input: column_name (string) = 34ff9752-4f40-4231-a777-1aeda30c862f
│                       │       └── input: table (table) = daa96129-0db7-4ddf-b2dd-4faf740bf212
│                       │           └── create.table
│                       │               └── input: file_bundle (file_bundle) = 78a4625d-03f6-431d-b174-d7ad3343f546
│                       │                   └── download.file_bundle
│                       │                       ├── input: sub_path (string) = ec21ca5d-2b3a-4aca-b75d-294d737e7f6b
│                       │                       └── input: url (string) = 8dd3c6a4-4f45-4b9d-a3e5-6a3e8526f3b6
│                       └── input: tokenize_by_word (boolean) = 5874bdf6-7f41-4199-92fb-8e9fdc5f7764
└── input: words_per_topic (integer) = a0d31bcc-d18c-4f29-989a-32177bc9c321